In [14]:
sys.path.append("..")
from utils import import_data
import numpy as np
x, y = import_data()
from torchvision.transforms import GaussianBlur
blurr = GaussianBlur(1, 0.1)
x = blurr(x[36]).numpy()
import cv2
cv2.imshow('img', x)
cv2.waitKey(0)
cv2.destroyAllWindows()
